# Simulation and Visualization of Diffusion Process with Path Integral

In [ ]:
# basic
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
# from numba_progress import ProgressBar
from tqdm import trange

# animation
from matplotlib import patches
from matplotlib.animation import ArtistAnimation
from IPython import display

$$
    P(x_i, y_j, t+\Delta t) 
    = 
    \sqrt{\frac{1}{4\pi D \Delta t}} 
    \sum_k 
    \exp \left[-  \frac{(x_i - x_k - u_x\Delta t)^2}{4D \Delta t} \right] 
    \Delta x 
    \cdot
    \sqrt{\frac{1}{4\pi D \Delta t}} 
    \sum_l
    \exp \left[-  \frac{(y_j - y_l - u_y\Delta t)^2}{4D \Delta t} \right] 
    \Delta y 
    \cdot 
    P(x_k, y_l, t)
$$

In [ ]:
# numerical solution
@njit(cache=True, nogil=True, parallel=True)
def PathIntegralP(
      P_prev: np.ndarray,
      x_list: np.ndarray,
      y_list: np.ndarray,
      delta_t: float,
      Dx: float,
      Dy: float,
      ux: float,
      uy: float,
      # progress_proxy: ProgressBar = None
    ):
    """path-integral
    P_prev : previous probability
    x_list : x value list of the target region
    y_list : y value list of the target region
    delta_t: descritized time step [s]
    Dx : diffusion coefficient in x axis [m^2/s]
    Dy : diffusion coefficient in y axis [m^2/s]
    ux : velocity in x axis [m/s]
    uy : velocity in y axis [m/s]
    """
    # constants
    x_size = x_list.size
    delta_x = (x_list[-1] - x_list[0]) / x_size
    y_size = y_list.size
    delta_y = (y_list[-1] - y_list[0]) / y_size
    scale = (1.0 / 4.0 * np.pi * delta_t) * np.sqrt(1.0 / (Dx * Dy)) * delta_x * delta_y

    # initialize
    P_next = np.zeros((x_size, y_size))

    for i in prange(x_size):
      for j in range(y_size):
        for k in range(x_size):
          for l in range(y_size):
            P_next[i, j] += scale * P_prev[k, l] * \
              np.exp(-(x_list[i] - x_list[k] - ux * delta_t)**2 - (y_list[j] - y_list[l] - uy * delta_t)**2)

      # progress_proxy.update(i)

    return P_next

In [ ]:
# simulation params
xmin, xmax = -5.0, +5.0 # [m]
delta_x = 0.1 # [m]
ymin, ymax = -5.0, +5.0 # [m]
delta_y = 0.1 # [m]
# make grid
x_list = np.linspace(xmin, xmax, int((xmax - xmin) / delta_x) + 1)
y_list = np.linspace(ymin, ymax, int((ymax - ymin) / delta_y) + 1)
# time settings
t0 = 0.0 # [sec]
t = 0.1 # [sec]
t_t0 = t - t0
delta_t = 0.1 # [s]

# physical parameters
Dx, Dy = 1.0, 2.0  # [m^2/s]
ux, uy = 1.0, 0.0 # [m/s]

# define prior distribution P_prev as a 2D gaussian distribution
x_mean, y_mean = 0.0, 0.0
x_sigma, y_sigma = 0.5, 0.5
P_prev = np.zeros((x_list.size, y_list.size))
for i in range(x_list.size):
  for j in range(y_list.size):
    P_prev[i, j] = np.exp(-((x_list[i] - x_mean)**2 / (2 * x_sigma**2) + (y_list[j] - y_mean)**2 / (2 * y_sigma**2))) / (2 * np.pi * x_sigma * y_sigma)

# make figure
fig, ax = plt.subplots()

# layout settings
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# set axis limits
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# draw P_prev
# ax.pcolormesh(x_list, y_list, P_prev.T, cmap='Blues')
ax.contour(x_list, y_list, P_prev.T, cmap='Blues')

# draw P(x) with fill & plot
# with ProgressBar(x_list.size) as progress_proxy:
#   P_next = PathIntegralP(P_prev, x_list, y_list, delta_t, Dx, Dy, ux, uy, progress_proxy)
P_next = PathIntegralP(P_prev, x_list, y_list, delta_t, Dx, Dy, ux, uy)
ax.pcolormesh(x_list, y_list, P_next.T, cmap='Blues')
ax.set_aspect('equal')

# show plot
plt.show()

In [ ]:
# prepare figure
fig, ax = plt.subplots(1, 1, figsize=(6,6))

# simulation params
xmin, xmax = -8.0, +8.0 # [m]
delta_x = 0.2 # [m]
ymin, ymax = -8.0, +8.0 # [m]
delta_y = 0.2 # [m]
# make grid
x_list = np.linspace(xmin, xmax, int((xmax - xmin) / delta_x) + 1)
y_list = np.linspace(ymin, ymax, int((ymax - ymin) / delta_y) + 1)
# time settings
t0 = 0.0 # [sec]
t = 0.0 # [sec]
delta_t = 0.1 # [s]
sim_steps = 70 # [steps]
# double wall params
wall_x_loc = 2.0 # [m]
wall_x_width = 1.0 # [m]
wall_x = [
          {
           "loc": wall_x_loc,
           "idx": int((wall_x_loc - xmin) / delta_x),
           "width": wall_x_width,
           "width_cell": int(wall_x_width / delta_x),
           }
          ]
wall_y_loc = 0.0 # [m]
wall_y_width = 0.5 # [m]
wall_y = [
          {
           "loc": wall_y_loc,
           "idx": int((wall_y_loc - ymin) / delta_y),
           "width": wall_y_width,
           "width_cell": int(1.0 / delta_x)
          }
         ]

wall_x_loc = 0.0 # [m]
slit_y_width = 1.0 # [m]
width_y = (ymax - ymin - 2.0 * slit_y_width) / 3.0
wall = [
    {
        "x": wall_x_loc, 
        "width_x": 0.5,
        "y": ymin + 2.0 * width_y + 2.0 * slit_y_width,
        "width_y": width_y,
    },
    {
        "x": wall_x_loc,
        "width_x": 0.5,
        "y": ymin + width_y + slit_y_width,
        "width_y": width_y,
    },
    {
        "x": wall_x_loc,
        "width_x": 0.5,
        "y": ymin,
        "width_y": width_y,
    },
]
# for each element in wall, add key "min_x_idx", "max_x_idx", "min_y_idx", "max_y_idx"
for w in wall:
    w["min_x_idx"] = int((w["x"]-xmin)/delta_x)
    w["max_x_idx"] = int((w["x"] + w["width_x"] - xmin) / delta_x)
    w["min_y_idx"] = int((w["y"]-ymin)/delta_y)
    w["max_y_idx"] = int((w["y"] + w["width_y"] - ymin) / delta_y)

# generate wall index list
wall_idx_list_x = []
wall_idx_list_y = []
for w in wall:
    for i in range(w["min_x_idx"], w["max_x_idx"]):
        for j in range(w["min_y_idx"], w["max_y_idx"]):
            wall_idx_list_x.append(i)
            wall_idx_list_y.append(j)

# physical parameters
Dx, Dy = 1.0, 2.0  # [m^2/s]
ux, uy = 2.0, 0.0 # [m/s]

# define prior distribution P_prev as a 2D gaussian distribution
x_mean, y_mean = -5.0, 0.0
x_sigma, y_sigma = 0.5, 0.5
P_now = np.zeros((x_list.size, y_list.size))
for i in range(x_list.size):
  for j in range(y_list.size):
    P_now[i, j] = np.exp(-((x_list[i] - x_mean)**2 / (2 * x_sigma**2) + (y_list[j] - y_mean)**2 / (2 * y_sigma**2))) / (2 * np.pi * x_sigma * y_sigma)

# graph layout settings
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_aspect('equal')
fig.tight_layout()

# simulation loop
frames = []
for i in trange(1, sim_steps):
    current_t = t0 + i * delta_t
    P_now = PathIntegralP(P_now, x_list, y_list, delta_t, Dx, Dy, ux, uy)
    # set P_now = 0.0 for the wall region
    for wx, wy in zip(wall_idx_list_x, wall_idx_list_y):
            P_now[wx, wy] = 0.0
    frame = [ax.pcolormesh(x_list, y_list, P_now.T*delta_x*delta_y, cmap='Blues')]
    text = "Time t = {t:>4.1f} [s]".format(t=current_t)
    frame += [ax.text(0.5, 0.9, text, ha='center', transform=ax.transAxes, fontsize=10, fontfamily='monospace')]
    frames.append(frame)

# show animation
ani = ArtistAnimation(fig, frames, interval=delta_t*1000)
html = display.HTML(ani.to_jshtml())
display.display(html)
plt.close()

# save animation
ani.save("path_integral_diffusion_2d_double_slit.mp4", writer="ffmpeg")